<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/07_14_22_GloVe_Conv1d_keras_gossipcop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For Baseline for RQ1. 

In [ ]:

!pip install wandb

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
import tensorflow as tf
import gensim.downloader as api

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from pathlib import Path

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output


metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/fakenews/ResearchFND")
assert base_dir.exists()

Mounted at /content/drive


In [ ]:
dataset_id = 'gossipcop'

In [ ]:
df = pd.read_csv(base_dir/f"{dataset_id}_agg.csv")
df.head()

,title,text,tweets,retweets,label,url
0,Kendall Kylie Jenner Jenner NOT Upset Up...,NaN,[],"['995423424741888001', '995461685166202880', '...",fake,NaN
1,Kim Kardashian Dethroned Dethroned By Khlo...,NaN,[],"['848843565027516416', '849030801970868224', '...",fake,NaN
2,Kim Kardashian Did NOT Hot Staffer Hot Sta...,NaN,[],"['940685393112064001', '977921622672920576', '...",fake,NaN
3,The Voice The Voice Team NOT Surprised Sur...,NaN,[],[],fake,NaN
4,Drake NOT Angelina Jolie s Toy Boy Toy Boy ...,NaN,"[{'id': 948630026496323585, 'text': 'Drake NOT...","['948022124626808832', '948630026496323585', '...",fake,NaN


In [ ]:
(df.title.str.len() == 0).any()

False

In [ ]:
len(df), len(df.title.unique())

(19968, 19968)

In [ ]:
news = df[['title', 'text', 'label']].copy()

print(news.isna().any())
news.count()

title    False
text      True
label    False
dtype: bool


title    19968
text     17691
label    19968
dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
news['title']= news['title'].apply(denoise_text)

In [ ]:
#Tokenizing Text -> Repsesenting each word by a number
tokenizer = text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(news.title)

In [ ]:
d_wv = 300
glove = api.load(f'glove-wiki-gigaword-{d_wv}')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
emb_matrix = np.zeros((max(tokenizer.word_index.values())+1, d_wv), dtype=np.float32)
for i in range(1,max(tokenizer.word_index.values())+1):
    word = tokenizer.index_word[i]
    if word in glove.vocab:
        emb_matrix[i, :] = glove.word_vec(word)

In [ ]:
len(tokenizer.word_index)

22625

In [ ]:
def get_model(emb_matrix):
    model = tf.keras.Sequential([
        #alternatively we can use glove.get_keras_embedding(),
        tf.keras.layers.Embedding(emb_matrix.shape[0], output_dim=emb_matrix.shape[1], weights=[emb_matrix],trainable=False, mask_zero=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(256, 5, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=5, strides=2),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dropout(0.2), 
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = get_model(emb_matrix)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         6787800   
                                                                 
 dropout (Dropout)           (None, None, 300)         0         
                                                                 
 conv1d (Conv1D)             (None, None, 256)         384256    
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 256)        0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, None, 32)          36992     
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                        

## Titles

In [ ]:
tokenized_titles = tokenizer.texts_to_sequences(news.title.tolist())
max_len = max(map(len, tokenized_titles))
X = sequence.pad_sequences(tokenized_titles, maxlen=max_len)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 25), (19968,))

In [ ]:
skf = StratifiedKFold(shuffle=True, random_state=124)

In [ ]:
def train(fold_id, train_idx, test_idx, params):

    # setup
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = get_model(emb_matrix)
    # training
    ckpt_filepath = f'ckpt/fold_{fold_id}'
    save_model_cb = keras.callbacks.ModelCheckpoint(
        filepath=ckpt_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test), 
        batch_size = params['bs'] , epochs=params['epochs'],
        callbacks=[WandbCallback(), save_model_cb]
    )
    #evaluation
    model.load_weights(ckpt_filepath)
    probs = model.predict(X_test).squeeze()
    y_pred = (probs > 0.5).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})
    return history

In [ ]:
params = {
    "epochs":200,
    "bs":128,
    # 'lr':1e-3
}

In [ ]:
GROUP = f"{dataset_id}-glove{d_wv}-title-convrnn"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="arampacha", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Epoch 1/200
125/125 [==============================] - 23s 133ms/step - loss: 0.4821 - accuracy: 0.7708 - val_loss: 0.4115 - val_accuracy: 0.8262 - _timestamp: 1657576867.0000 - _runtime: 27.0000
Epoch 2/200
125/125 [==============================] - 16s 126ms/step - loss: 0.3868 - accuracy: 0.8307 - val_loss: 0.3937 - val_accuracy: 0.8307 - _timestamp: 1657576883.0000 - _runtime: 43.0000
Epoch 3/200
125/125 [==============================] - 15s 123ms/step - loss: 0.3281 - accuracy: 0.8620 - val_loss: 0.3946 - val_accuracy: 0.8443 - _timestamp: 1657576898.0000 - _runtime: 58.0000
Epoch 4/200
125/125 [==============================] - 18s 146ms/step - loss: 0.2586 - accuracy: 0.8938 - val_loss: 0.4173 - val_accuracy: 0.8235 - _timestamp: 1657576917.0000 - _runtime: 77.0000
Epoch 5/200
125/125 [==============================] - 15s 122ms/step - loss: 0.1874 - accuracy: 0.9246 - val_loss: 0.4786 - va

accuracy,▁▆▇█████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,█▆▇▅▆▅▂▅▆▆▁▇▄▅▆▇▅▆▄▆▅▄▄▆▆▃▆▄▅▆▅▇▅▅▃▆▆▅▆▆
val_loss,▁▂▄▄▅▅▆▅▅▆▆▆▆▆▆▆▆▆▆▇█▇▆▅▇▇█▇▇▆▇▆▇▆▇▆▆▇▇▆
GFLOPS,0.0
accuracy,0.99743


## Fulltext

In [ ]:
news['text'].fillna('', inplace=True)
news['text']= news['text'].apply(denoise_text)

news['fulltext'] = news['title'] + '\n' + news['text']

In [ ]:
#Tokenizing Text -> Repsesenting each word by a number
tokenizer = text.Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(news.fulltext)

In [ ]:
emb_matrix = np.zeros((len(tokenizer.word_index)+1, d_wv), dtype=np.float32)
for i in range(1,max(tokenizer.word_index.values())+1):
    word = tokenizer.index_word[i]
    if word in glove.vocab:
        emb_matrix[i, :] = glove.word_vec(word)
print(emb_matrix.shape)

(155478, 300)


In [ ]:
tokenized_texts = tokenizer.texts_to_sequences(news.fulltext.tolist())
max_len = 128
X = sequence.pad_sequences(tokenized_texts, maxlen=max_len)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 128), (19968,))

In [ ]:
params = {
    "epochs":200,
    "bs":128,
    # 'lr':1e-3
}

In [ ]:
GROUP = f"{dataset_id}-glove{d_wv}-fulltext{max_len}-convnet"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity="arampacha", project="fnd", group=GROUP, name=f"{GROUP}-fold-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

wandb: Currently logged in as: arampacha. Use `wandb login --relogin` to force relogin


Epoch 1/200
125/125 [==============================] - 97s 745ms/step - loss: 0.4927 - accuracy: 0.7705 - val_loss: 0.4274 - val_accuracy: 0.8117 - _timestamp: 1657580195.0000 - _runtime: 108.0000
Epoch 2/200
125/125 [==============================] - 89s 710ms/step - loss: 0.4034 - accuracy: 0.8306 - val_loss: 0.4075 - val_accuracy: 0.8315 - _timestamp: 1657580284.0000 - _runtime: 197.0000
Epoch 3/200
125/125 [==============================] - 84s 669ms/step - loss: 0.3522 - accuracy: 0.8553 - val_loss: 0.4015 - val_accuracy: 0.8322 - _timestamp: 1657580368.0000 - _runtime: 281.0000
Epoch 4/200
125/125 [==============================] - 83s 667ms/step - loss: 0.3097 - accuracy: 0.8712 - val_loss: 0.4390 - val_accuracy: 0.8272 - _timestamp: 1657580453.0000 - _runtime: 366.0000
Epoch 5/200
125/125 [==============================] - 82s 657ms/step - loss: 0.2380 - accuracy: 0.9049 - val_loss: 0.4597 - val_accuracy: 0.8157 - _timestamp: 1657580535.0000 - _runtime: 448.0000
Epoch 6/200
125

accuracy,▁▅▇█████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▇▄▁▃▆█▅▆▄▄▃▇▆▆▅▆▇▆▄▆▇▄▄▇▂▆▇▆▃▇▆▅▆▆▆▅▇▇▄▄
val_loss,▁▂▃▄▅▅▅▅▅▅▆▆▆▆▇▇▇▆▆▆▆▇▆█▆▆▆▇▆▇▇▆█▇▆▇███▇
GFLOPS,0.0
accuracy,0.99718
